# Importing required libraries


In [ ]:
!unzip SubtitlesFinal.zip

In [25]:
import pandas as pd
import os
import spacy

#Reading .csv Files from the Folder

In [6]:
folder_pathu = '/content/Subtitles Final/Top Rated'  # Specify the folder path where the CSV files are located

for i in range(1, 31):
    filename = f"dfu{i}.csv"  # Create the filename based on the DataFrame index
    file_pathu = os.path.join(folder_pathu, filename)  # Create the complete file path

    # Check if the file exists before attempting to read it
    if os.path.isfile(file_pathu):
        df_name = f"dfu{i}"  # Create the DataFrame name based on the index
        globals()[df_name] = pd.read_csv(file_pathu)  # Read the CSV file into a DataFrame



In [8]:
folder_pathm = '/content/Subtitles Final/Middle Rated'  # Specify the folder path where the CSV files are located

for i in range(1, 31):
    filename = f"dfm{i}.csv"  # Create the filename based on the DataFrame index
    file_pathm = os.path.join(folder_pathm, filename)  # Create the complete file path

    # Check if the file exists before attempting to read it
    if os.path.isfile(file_pathm):
        df_name = f"dfm{i}"  # Create the DataFrame name based on the index
        globals()[df_name] = pd.read_csv(file_pathm)  # Read the CSV file into a DataFrame



In [12]:
folder_pathl = '/content/Subtitles Final/Low Rated'  # Specify the folder path where the CSV files are located

for i in range(1, 31):
    filename = f"dfl{i}.csv"  # Create the filename based on the DataFrame index
    file_pathl = os.path.join(folder_pathl, filename)  # Create the complete file path

    # Check if the file exists before attempting to read it
    if os.path.isfile(file_pathl):
        df_name = f"dfl{i}"  # Create the DataFrame name based on the index
        globals()[df_name] = pd.read_csv(file_pathl)  # Read the CSV file into a DataFrame



#Category Wise List Of DataFrames

In [14]:
upper=[dfu1,dfu2,dfu3,dfu4,dfu5,dfu6,dfu7,dfu8,dfu9,dfu10,dfu11,dfu12,dfu13,dfu14,dfu15,dfu16,dfu17,dfu18,dfu19,dfu20,dfu21,dfu22,dfu23,dfu24,dfu25,dfu26,dfu27,dfu28,dfu29,dfu30]
middle=[dfm1,dfm2,dfm3,dfm4,dfm5,dfm6,dfm7,dfm8,dfm9,dfm10,dfm11,dfm12,dfm13,dfm14,dfm15,dfm16,dfm17,dfm18,dfm19,dfm20,dfm21,dfm22,dfm23,dfm24,dfm25,dfm26,dfm27,dfm28,dfm29,dfm30]
lower=[dfl1,dfl2,dfl3,dfl4,dfl5,dfl6,dfl7,dfl8,dfl9,dfl10,dfl11,dfl12,dfl13,dfl14,dfl15,dfl16,dfl17,dfl18,dfl19,dfl20,dfl21,dfl22,dfl23,dfl24,dfl25,dfl26,dfl27,dfl28,dfl29,dfl30]


In [15]:
def time_taken(l):
  for i in range(len(l)):
    l[i]['TimeTaken'] = (l[i]['EndMilliseconds'] - l[i]['StartMilliseconds']) / 1000
time_taken(middle)
time_taken(lower)
time_taken(upper)

## Segmentation Approach 1

In [20]:
def segmentation_1(df, threshold):
    merged_text = []
    time_taken = []
    scene_number = []
    num_words = []
    current_text = ''
    current_time = 0
    current_scene = 1
    current_words = 0

    for i, row in df.iterrows():
        text = row['Text']
        time = row['TimeTaken']
        words = len(text.split())

        if current_time + time <= threshold:
            current_text += ' ' + text
            current_time += time
            current_words += words
        else:
            merged_text.append(current_text.strip())
            time_taken.append(current_time)
            scene_number.append(current_scene)
            num_words.append(current_words)
            current_text = text
            current_time = time
            current_scene += 1
            current_words = words

    merged_text.append(current_text.strip())
    time_taken.append(current_time)
    scene_number.append(current_scene)
    num_words.append(current_words)

    merged_df = pd.DataFrame({
        'Scene': scene_number,
        'Text': merged_text,
        'TimeTaken': time_taken,
        'NumberOfWords': num_words
    })

    return merged_df


## Segmentation Approach 2

In [19]:
def segmentation_2(df, num_scenes):
    sentences_per_row = len(df) // num_scenes

    scenes = []
    texts = []
    time_taken = []
    num_words = []  # Added list to store the number of words

    scene_number = 1

    for i in range(0, len(df), sentences_per_row):
        scene_texts = df['Text'].iloc[i:i+sentences_per_row].tolist()
        scene_text = ' '.join(scene_texts)

        scenes.append(scene_number)
        texts.append(scene_text)
        time_taken.append(df['TimeTaken'].iloc[i:i+sentences_per_row].sum())

        # Count the number of words in the merged text
        num_words.append(len(scene_text.split()))

        scene_number += 1

    df_merged = pd.DataFrame({
        'Scene': scenes,
        'Text': texts,
        'TimeTaken': time_taken,
        'NumberOfWords': num_words  # Added 'NumberOfWords' column
    })

    return df_merged

In [21]:
def segmentdict1(l):
  merged = {}
  for i in range(len(l)):
      new_dfname = f"a1_df{i+1}"
      new_df = segmentation_1(l[i], 90)
      merged[new_dfname]=new_df
  return merged

def segmentdict2(l):
  merged = {}
  for i in range(len(l)):
      new_dfname = f"a2_df{i+1}"
      new_df = segmentation_2(l[i], 110)
      merged[new_dfname]=new_df
  return merged

In [22]:
a1_middle=segmentdict1(middle)
a1_lower=segmentdict1(lower)
a1_upper=segmentdict1(upper)

a2_middle=segmentdict2(middle)
a2_lower=segmentdict2(lower)
a2_upper=segmentdict2(upper)

## Text Cleaning

In [26]:
!pip install contractions -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.2 MB/s eta 0:00:00


In [27]:
import contractions

def cleaning(d):
  nlp=spacy.load("en_core_web_sm")
  def pos_removal(s):
      doc = nlp(s)
      modified = ' '.join([str(token) for token in doc if token.pos_ not in ['DET', 'NUM','PUNCT', 'PROPN','SYM']])
      return modified
  contractions.add(['bruv','bro','bruh'], 'brother')
  contractions.add('salty','jealous')
  contractions.add(['ma','mom'],'mother')
  contractions.add(['pops','papa'],'father')
  contractions.add(['nuts'],'crazy')
  def slang(s):
    text = contractions.fix(s, slang=True)
    return text
  for i in d:
    d[i]['Text'] = d[i]['Text'].str.replace('♪', '')
    d[i]['Text'] = d[i]['Text'].str.replace('\n', ' ')
    d[i]['clean_text']=d[i]['Text'].apply(slang)
    d[i]['clean_text']=d[i]['clean_text'].apply(pos_removal)
  return d

In [28]:
a1_middle=cleaning(a1_middle)
a1_lower=cleaning(a1_lower)
a1_upper=cleaning(a1_upper)

a2_middle=cleaning(a2_middle)
a2_lower=cleaning(a2_lower)
a2_upper=cleaning(a2_upper)

# Emotion Prediction using EmoRoBERTa Model availabel on Hugging Face

In [29]:
!pip install transformers -q
from transformers import pipeline


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.0 MB/s eta 0:00:00


In [30]:
emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [31]:
def get_emotion_label(text):
  return(emotion(text)[0]['label'])
def emotion_label(dic):
  for i in dic:
    dic[i]['emotion']=dic[i]['clean_text'].apply(get_emotion_label)

# These Files are saved in Prediction Emotion Folder

In [ ]:
emotion_label(a1_middle)
emotion_label(a1_lower)
emotion_label(a1_upper)

emotion_label(a2_middle)
emotion_label(a2_lower)
emotion_label(a2_upper)



The above code will take some time, you can find the predicted files in Predicted Emotin folder for both the approaches

Modiofy Below code to save files as per your requirements

In [ ]:
import os
target_folder_path = '/content/Predicted Emotions/approach-1'

# Create the target folder (if it doesn't exist)
if not os.path.exists(target_folder_path):
    os.makedirs(target_folder_path)

# Save each DataFrame as a CSV file in the Google Drive folder
for i in emotion_label(a1_middle):
    submission3 = pd.DataFrame(upper_merged_clean[i])
    csv_filename = f'{i}u.csv'
    csv_filepath = os.path.join(target_folder_path, csv_filename)

    # Save the DataFrame as a CSV file
    submission3.to_csv(csv_filepath, index=False)

    # Print the saved file path
    print(f"CSV file saved: {csv_filepath}")